# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузим и подготовим данные.
2. Обучим разные модели. 
3. Сделаем выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [20]:
#Импорт библиотек
import pandas as pd
import numpy as np
from scipy import stats as st

#хитмэп корреляции
import os
import seaborn as sns #построение графиков

#операции с выборкой
from sklearn.model_selection import train_test_split


#сохранение модели
import joblib 
from joblib import dump

#классификация
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 

#регрессия
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor #создание константной модели (среднее, медиана и др)

#преобразователь масштаба методом стандартизации
from sklearn.preprocessing import StandardScaler

#метрики
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score #кросс-валидация
from sklearn.metrics import make_scorer #для написания ручной оценки

#подбор параметров
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#построение ROC-кривой
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve 
from sklearn.metrics import roc_auc_score

#раздел upsampling
from sklearn.utils import shuffle

#библиотека LightGBM
import lightgbm as lgb
from lightgbm import LGBMRegressor

#библиотека CatBoost
import catboost as cb
from catboost import CatBoostRegressor

#разложение временного ряда на тренд и сезонную компоненту, построение графиков этих составляющих ряда
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

#замена стандартной кросс-валидации для временных рядов
from sklearn.model_selection import TimeSeriesSplit

#для графиков
import pylab
#plt.style.use('ggplot')

#оценка тональности текстов
import torch # %pip install torchvision
import transformers as ppb # %pip install transformers
from tqdm import notebook
from sklearn.model_selection import cross_val_score

In [21]:
#загрузим датасет
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')   
except FileNotFoundError:
    data = pd.read_csv('toxic_comments.csv')
    
data

In [22]:
data_bert1 = data.sample(100, random_state=12345).reset_index(drop=True)
data_bert2 = data.sample(3000, random_state=12345).reset_index(drop=True)

#shorted_512 = data['lemm_text'].apply(lambda x: re.sub(r'^(.{512}).*$', '\g<1>', x))

### Вывод
    Данные загружены, 159570 строк, столбец text содержит текст комментариев на английском, а toxic — целевой признак.

## Обучение

### BERT v1 

In [23]:
#Importing pre-trained DistilBERT model and tokenizer
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

#Importing pre-trained BERT model and tokenizer:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

#Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [24]:
tokenized = data_bert1['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

In [25]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [26]:
input_ids = torch.tensor(np.array(padded))

with torch.no_grad():
    last_hidden_states = model(input_ids)

In [27]:
# Slice the output for the first position for all the sequences, take all hidden unit outputs
features = last_hidden_states[0][:,0,:].numpy()

In [28]:
labels = data_bert1['toxic']
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.1, random_state=12345)

In [29]:
#train the Logistic Regression model on the training set.
lr_clf = LogisticRegression(random_state=12345, max_iter=1000)
lr_clf.fit(train_features, train_labels)

In [30]:
#Now that the model is trained, we can score it against the test set
predictions_train = lr_clf.predict(train_features) 
predictions_test = lr_clf.predict(test_features)

f1_train = f1_score(train_labels, predictions_train)
f1_test = f1_score(test_labels, predictions_test)

print(f1_train, f1_test)

### BERT v2

In [13]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

#tokenizer = transformers.BertTokenizer(vocab_file='/datasets/ds_bert/vocab.txt')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

tokenized = data_bert2['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [14]:
#config = transformers.BertConfig.from_json_file('/datasets/ds_bert/bert_config.json')
#model = transformers.BertModel.from_pretrained('/datasets/ds_bert/rubert_model.bin', config=config)
model = model_class.from_pretrained(pretrained_weights)

In [15]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [16]:
# обучим и протестируем модель
target = data_bert2['toxic']
features = np.concatenate(embeddings)

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=12345)

In [17]:
print(target_train.shape, target_test.shape, features_train.shape, features_test.shape)

In [18]:
model = LogisticRegression(random_state=12345, max_iter=10000)
model.fit(features_train, target_train) 

predictions_train = model.predict(features_train) 
predictions_test = model.predict(features_test)

f1_train = f1_score(target_train, predictions_train)
f1_test = f1_score(target_test, predictions_test)
print(f1_train, f1_test)

### Вывод
    На версии BERT 2 достигнуто целевое значение метрики f1 0,75.

## Общий вывод
    Первая версия взята из источника http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
    При первых запусках показывала f1 около 0,25, при следующих запусках - 0.0.
    
    С помощью второй версии из теории получили метрику f1 = 0.77 согласно цели проекта. 
    BERT ресурсоемкая модель, поэтому получилось запустить (при 12Gb ОЗУ, если выбираем больше 3000 - то ошибка нехватки памяти) проект и достить нужного значения метрики - благодаря размеру sample 3000 в сочетании с test_size=0.1. При большем размере sample, метрика f1 вероятно будет выше. Начинал с sample 500, повышая каждый раз на 500 размер выборки, прирост f1 с каждым увеличением выборки - 3-5%.
    
    Для второй версии несяно где брать 2 параметра: 
    #config = transformers.BertConfig.from_json_file('/datasets/ds_bert/bert_config.json')
    и
    #model = transformers.BertModel.from_pretrained('/datasets/ds_bert/rubert_model.bin', config=config)
    Эти 2 параметра взяты из первого кода BERT, неясно насколько это корректно.
    
    Для повышения результатов метрики возможно проверить и устранить дисбаланс классов и повысить количество samples.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны